In [1]:
# Last inn 'X_train_LSTM' fra en CSV-fil
X_df = pd.read_csv(url_data+'X_tensorflow.csv')
# Konverter hele DataFrame til et NumPy array
X = X_df.to_numpy()

# Last inn 'y_train_LSTM' fra en CSV-fil
y_df = pd.read_csv(url_data+'y_liberal.csv')
# Konverter hele DataFrame til et NumPy array
y = y_df.to_numpy()

# Last inn 'y_train_LSTM' fra en CSV-fil
embeddings_GloVe = pd.read_csv(url_data+'embeddings_glove.csv')
# Konverter hele DataFrame til et NumPy array
embeddings_GloVe = embeddings_GloVe.to_numpy()

print('Shape of label tensor:', y.shape)
print('Shape of X:', X.shape)
print('Shape of embeddings_GloVe:', embeddings_GloVe.shape)

NameError: name 'pd' is not defined

## LSTM

In [ ]:
tf.random.set_seed(42)

#batch_size = None
#batch_size=64

model = Sequential()
model.add(Input(shape=(20,)))
# Adding the Embedding layer with pre-trained weights and specifying input_length
model.add(Embedding(input_dim=embeddings_GloVe.shape[0],
                    output_dim=embeddings_GloVe.shape[1],
                    weights=[embeddings_GloVe],
                    trainable=False))

model.add(Bidirectional(LSTM(64, # Number of hidden states, number of reccurent units for each vector
    activation='tanh',
    recurrent_activation='sigmoid',
    recurrent_dropout=0.2,
    return_sequences=False)))

model.add(Dense(77, activation='relu'))
model.add(Dropout(0.20))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model summary
model.summary()

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)


# history = model.fit(X_train_LSTM, y_train_LSTM,
#                     epochs=20, # Adjust based on your needs
#                     validation_split=0.2,
#                     batch_size=64,
#                     callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


In [ ]:
# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project="text_classification")

# Function to train the model
def train():
    # Initialize a new wandb run
    wandb.init(reinit=True)
    
    hypermodel = TextClassifier_WB(input_length, embeddings_GloVe, num_classes, parallel_blocks)
    model = hypermodel.build()
    
    # Train the model
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, callbacks=[WandbCallback()])
    wandb.finish()

# Run the sweep
wandb.agent(sweep_id, train)

In [ ]:
import logging
logging.getLogger('matplotlib.font_manager').disabled = True

with plt.xkcd():
    plt.plot(model_1.history['accuracy'], label='train_acc')
    plt.plot(model_1.history['val_accuracy'], label='val_acc')
    plt.plot(model_1.history['loss'], label='train_loss')
    plt.plot(model_1.history['val_loss'], label='val_loss')

    plt.xlabel('Epoch #')
    plt.ylabel('Value')
    plt.title('Training results on initial model')
    plt.legend()

    plt.show()

## With POS features

In [ ]:
from keras.models import Model
from keras.layers import Input, Embedding, SpatialDropout1D, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout, concatenate

# Text input pathway
text_input = Input(shape=(max_len,), dtype='int32', name='text_input')  # Ensure the shape parameter matches your input length
text_embedded = Embedding(input_dim=vocab_size+1,
                          output_dim=embedding_dim,
                          input_length=max_len,
                          weights=[embeddings_GloVe],
                          trainable=False)(text_input)
text_dropout = SpatialDropout1D(0.2)(text_embedded)
text_conv = Conv1D(filters=32, kernel_size=5, activation='relu')(text_dropout)
text_pool = MaxPooling1D(pool_size=4)(text_conv)
text_lstm = Bidirectional(LSTM(64))(text_pool)

# POS tags input pathway
pos_input = Input(shape=(max_len,), dtype='int32', name='pos_input')
pos_embedded = Embedding(input_dim=POS_size+1, output_dim=100, input_length=max_len)(pos_input)
pos_dropout = SpatialDropout1D(0.2)(pos_embedded)
pos_conv = Conv1D(filters=32, kernel_size=5, activation='relu')(pos_dropout)
pos_pool = MaxPooling1D(pool_size=4)(pos_conv)
pos_lstm = Bidirectional(LSTM(32))(pos_pool)

# Combine the pathways
combined = concatenate([text_lstm, pos_lstm])

# Classification layers
layer = Dense(128, activation='relu')(combined)
layer = Dropout(0.5)(layer)
output = Dense(2, activation='softmax')(layer)

# Build and compile the model
POS_text_model = Model(inputs=[text_input, pos_input], outputs=output)
POS_text_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], run_eagerly=False)

# Model summary
POS_text_model.summary()

In [ ]:
with plt.xkcd():
    plt.plot(POS_text_model.history['accuracy'], label='train_acc')
    plt.plot(POS_text_model.history['val_accuracy'], label='val_acc')
    plt.plot(POS_text_model.history['loss'], label='train_loss')
    plt.plot(POS_text_model.history['val_loss'], label='val_loss')

    plt.xlabel('Epoch #')
    plt.ylabel('Value')
    plt.title('Training results on initial model')
    plt.legend()

    plt.show()